<div class="alert alert-block alert-success">
    
## Python libraries used by this module
    
</div>

In [ ]:
%matplotlib inline
from sys import path as spath
spath.append("scripts/") #Location of the modules created for this pipeline

#Modules created for this pipeline
from CRISPRtools import *
from easyFunctions import *
from HMMParser import *

#Native and conda installed modules
import re
from Bio.SeqIO import parse, write
from collections import Counter
from datetime import date
from glob import glob
from IPython.display import display, HTML
from matplotlib import pyplot as plt
from os import chdir, listdir, path, stat, system, walk
from pandas import Series
from pickle import load

<div class="alert alert-block alert-danger">
    
## Designate where the fasta files are located and the valid file extensions
#### The fasta directory will be recursively searched for all files with the given extensions
    
</div>

In [ ]:
baseDbsDir = "/mnt/research/germs/shane/databases/assemblies/"
validExts = set([".fasta",".fna",".fa"])

<div class="alert alert-block alert-success">

## Load the genomic assemblies
    
</div>

In [ ]:
def baseAssembly(fname): return fname[:fname.rfind(".")], fname[fname.rfind("."):]
def deepWalk(baseDir,validAsmExts,visited=set()):
    visited.add(baseDir)
    for (dirpath, dirnames, filenames) in walk(baseDir):
        for assembly in filenames:
            fPath = path.join(dirpath, assembly)
            asmID, ext = baseAssembly(assembly)
            if ext not in validAsmExts or fPath in visited:continue
            visited.add(fPath)
            yield fPath
        for dirName in dirnames:
            dPath = path.join(baseDir,dirName)
            if dPath not in visited: 
                for f in deepWalk(dPath, validAsmExts, visited): yield f         
allAssemblies = {}
for asmPath in deepWalk(baseDbsDir,validExts):
    asmID = asmPath[asmPath.rfind("/")+1:]
    allAssemblies[asmID] = asmPath

theDate = date.today()
print("Ready to dump")
dump(allAssemblies,open("pickles/allAssemblies.p","wb"))     
print("Number of assemblies on %s: %i" % (theDate, len(allAssemblies)))

<div class="alert alert-block alert-success">

## Build the scripts to search for CRISPR arrays using both MinCED and PilerCR

</div>

In [ ]:
%%bash
python ../scripts/FindCRISPRs.py --fasta_dir pickles/allAssemblies.p --scriptsDir ../scripts/findCRISPRs

<div class="alert alert-block alert-success">
    
## Process the CRISPR results
1. Check to see if the assembly related to file has a crispr array
1. Add any assembly with a crispr array to a master list

</div>

In [ ]:
crisprDir = "/mnt/research/germs/shane/databases/crisprs/"
tools = ["pilerCR/", "minCED/"]
dbs = ['pat2','genbank','refseq']
crisprs = CasOperons()
for tool in tools:
    for db in dbs:
        filePath = crisprDir+tool+db+"/"
        crisprs.hasCrispr(listdir(filePath), tool == "pilerCR/", filePath, allAssemblies)
crisprs.saveProgress()
dump(crisprs, open("pickles/CRISPRs.p","wb")) 

<div class="alert alert-block alert-success">
    
# From the assemblies with a CRISPR, find assemblies that have a Cas9-like coding sequence
## Step 1: Profile the Protein 
1. Mean, max, min length of already known orthologs
    
## Step 2: HMM Seach 
1. Align orthologs
1. build hmm profile
1. hmm search against a database  
    
## Step 3: Profile Results  
1. Read hmm results
1. cluster results by percent identity
1. remove proteins without required domains
    
</div>

<div class="alert alert-block alert-danger">
    Protein profile input
</div>

In [ ]:
protein = "Cas9"
#File containing the seed proteins for the hmmsearch
proteinProfile = "proteins/DiverseCas9s.faa" 

In [ ]:
#Output files and directories
proteinFile = "proteins/%s.faa" % protein
alnName  = "alignments/%s.aln" % protein
hmmName  = "hmm/%s.hmm" % protein
hmmResultsDir = "hmm/results"
refDatabases = ["NCBI/refseq/bacteria","NCBI/refseq/archaea","NCBI/genbank/bacteria","NCBI/genbank/archaea","PATRIC2/fastas","Corteva"]

<div class="alert alert-block alert-success">

## Profile the protein sequence
    
</div>

In [ ]:
dists = []
for rec in parse(proteinProfile,"fasta"): dists.append(len(rec.seq))
dists = Series(dists)
minSeqLen = int(dists.min() - (dists.mean() * .25))
maxSeqLen = int(dists.max() + (dists.mean() * .25))
print (dists.describe());
print ("Finding proteins that match the protein profile and are between %i and %i" % (minSeqLen, maxSeqLen))
import matplotlib.pyplot as plt
dists.plot(kind='hist',grid=True,title="Diverse Cas9 Length Distribution",figsize=(6,4))
plt.savefig("images/DiverseCas9Lengths.png")
plt.show()

<div class="alert alert-block alert-success">

## Align and build a profile HMM from the coding sequence
    
</div>

In [ ]:
%%bash -s $geneProfile $alnName $hmmName
mafft --thread 15 --maxiterate 1000 --retree 100 --localpair --reorder --treeout $1 > $2
hmmbuild $3 $2 2>&1 >/dev/null

<div class="alert alert-block alert-success">

## Build the scripts to search for sequences that match the HMM
    
</div>

In [ ]:
system("rm ../scripts/hpc/hmmSearch/*")
print("There are %i CRISPR related assemblies" % (len(assembliesWCrisprs)))
fh = open("../scripts/hpc/hmmSearch/HMMSearch_0.sb","w")
header = ""
for line in open("../scripts/hpc/header.sb"): header+=line
fh.write(header)
cmdCount, fileCounter = 0,1
getOrfsCMD = "python /mnt/research/germs/shane/transActRNA/scripts/GetOrfs.py %s sequences/orfs/%s.orfs 673 >/dev/null"
hmmSearchCMD = "hmmsearch hmm/%s.hmm sequences/orfs/%s.orfs >%s/%s.hmmout"

for asmName, operon in assembliesWCrisprs.items():
    asmFilePath = operon.assembly
    if "%s/%s.hmmout" % (hmmResultsDir,asmName) in hmmFiles: continue
    cmdCount += 1
    fh.write("if [ ! -f %s/%s.hmmout ]; then\n" % (hmmResultsDir,asmName)) #Check to see if the hmm profile already exists
    fh.write("\t"+getOrfsCMD % (asmFilePath,asmName) + "\n")
    fh.write("\t"+hmmSearchCMD % (gene,asmName,hmmResultsDir,asmName)+"\n")
    fh.write("\thmmsize=$(wc -l <\"%s/%s.hmmout\") \n" % (hmmResultsDir,asmName))
    fh.write("\tif [ $hmmsize -le 40 ]; then \n")
    fh.write("\t\trm sequences/orfs/%s.orfs \n" % (asmName))
    fh.write("\tfi\n")
    fh.write("fi\n\n")
    if cmdCount % 50 == 0:
        fh.close()
        fh = open("../scripts/hpc/hmmSearch/HMMSearch_%i.sb" % (fileCounter),"w")
        fh.write(header)
        fileCounter += 1
fh.close()
print("There are %i files for %i assemblies" % (fileCounter,cmdCount))

<div class="alert alert-block alert-success">

## Launch HMM scripts built in the previous step

</div>

In [ ]:
%%bash 
bash ../scripts/hpc/LaunchHMMSearch.sh

<div class="alert alert-block alert-success">

## Read the HMM results and create protein and nucleotide sequence files

</div>

In [ ]:
crisprFiles = load(open("pickles/CRISPRs.p","rb")) 
casOperons = CasOperons(protein)
casOperons.hasCas9(hmmResultsDir+"/",crisprFiles)
dump(casOperons, "pickles/%s_Operons.p" % protein)

# Sort through all the results and make a file with all of the Cas proteins of interest. 
# If there are duplicates only keep the duplicates that come from a unique chromosome
#casOperons = load(open("pickles/%s_Operons.p" % protein,"rb"))
allCasAsmFile = "assemblies/All_%s_Representative_Assemblies.fasta" % (protein)
allCasAAsFile = "proteins/All_%s-Like.faa" % (protein)
casOperons.uniqueNukeSeqs(allCasAsmFile,allCasAAsFile,protein)

<div class="alert alert-block alert-success">

## Domain search on the proteins matching the HMM.

</div>

In [ ]:
%%bash
sbatch ../scripts/hpc/DomainSearch.sb

<div class="alert alert-block alert-success">

## Read the results from the domain search.

</div>

In [ ]:
spacePat = re.compile( r'^[\t ]*$')
hits=SamplesDict()
print("Reading Table Definitions")
for line in open("hmm/Cas9-Like_phi.faa.domtbl"):
    if line.startswith( '#' ):continue
    fields = re.split( r'(\[[^\[]*[^\S]+[^\]]*\]|[^\t ]+)', line.strip() )
    fields = DomainHit(list(filter(lambda i: not spacePat.search(i), fields)))
    hits[fields.hit]=fields
print(len(hits.samples))
dump(hits,"pickles/%s_HMM_DOMAIN_Search_Results.p" % (protein))
hasAllDomains = hits["RuvC_1_Cas9"].intersection(hits["RuvC_2_Cas9"].intersection(hits["RuvC_3_Cas9"].intersection(hits["HNH_Cas"])))
allSamples = set(hits.samples.keys())
nSamples = len(allSamples)
noRuvC1 = allSamples.difference(hits["RuvC_1_Cas9"])
noRuvC2 = allSamples.difference(hits["RuvC_2_Cas9"])
noRuvC3 = allSamples.difference(hits["RuvC_3_Cas9"])
noHNH   = allSamples.difference(hits["HNH_Cas"])
htmlString ="""<table align='left'>
    """\
    "<tr style='background-color:#9B7E46;color:white'><td>Number of Proteins:</td><td>%i</td></tr>" % (nSamples) +\
    "<tr style='background-color:#373D20;color:white'><td>Has all domains</td><td>%i</td></tr>" % (len(hasAllDomains)) +\
    "<tr style='background-color:#BCBD8B;color:white'><td>Number of sequences with no detected RuvCI domain</td><td>%i</td></tr>" % (len(noRuvC1)) +\
    "<tr style='background-color:#717744;color:white'><td>Number of sequences with no detected RuvCII domain</td><td>%i</td></tr>" % (len(noRuvC2)) +\
    "<tr style='background-color:#766153;color:white'><td>Number of sequences with no detected RuvCIII domain</td><td>%i</td></tr>" % (len(noRuvC3)) +\
    "<tr style='background-color:#F4B266;color:white'><td>Number of sequences with no detected HnH</td><td>%i</td></tr>" % (len(noHNH)) +\
    """
    </table>
"""
display(HTML(htmlString))

<div class="alert alert-block alert-success">

## Remove sequences with a domain that is not in the right location

</div>

In [ ]:
ruvC1Coords = []
ruvC3Coords = []
samples = []
difs = []
for sample in hasAllDomains:
    ruc1Start = hits.samples[sample]["RuvC_1_Cas9"].start
    ruc3End = hits.samples[sample]["RuvC_3_Cas9"].dend
    ruvC1Coords.append(ruc1Start)
    ruvC3Coords.append(ruc3End)
    difs.append(ruc3End-ruc1Start)
    samples.append(sample)
    
ruvC1Dists = Series(ruvC1Coords,index=samples)
ruvC3Dists = Series(ruvC3Coords,index=samples)
distBetween = Series(difs,index=samples)
print ("\nMean distance from RuvC_1 to start of protein: %.0f std: %.0f" % (ruvC1Dists.mean(), ruvC1Dists.std()))
# the histogram of the data
n, bins, patches = plt.hist(ruvC1Dists,facecolor='green')
plt.xlabel('Distance to start of protein')
plt.ylabel('Number of sequences')
plt.title("Histogram of distances from RuvC_1 Domain to start of protein")
plt.grid(True)
plt.show()

In [ ]:
print ("\nMean distance from RuvC_3 to end of protein: %.0f std: %.0f" % (ruvC3Dists.mean(), ruvC3Dists.std()))
# The histogram of the data
n, bins, patches = plt.hist(ruvC3Dists,facecolor='blue')
plt.xlabel('Distance To End')
plt.ylabel('Number of sequences')
plt.title("Distance From RuvC_3 Domain to End of Sequence")
plt.grid(True)
plt.savefig("images/%s_RuvC3_DomainLengths.png" % gene)
plt.show()

In [ ]:
start_outliers = set(ruvC1Dists[ruvC1Dists > ruvC1Dists.mean()+8*ruvC1Dists.std()].index) #RuvC1 outlier
end_outliers = set(ruvC3Dists[ruvC3Dists > ruvC3Dists.mean()+5*ruvC3Dists.std()].index) #RuvC3 outlier
hasGoodDomains = hasAllDomains.difference(start_outliers.union(end_outliers))
htmlString ="""<table align='left'>
    """\
    "<tr style='background-color:#373D20;color:white'><td>No outlier domains</td><td>%i</td></tr>" % (len(hasGoodDomains)) +\
    "<tr style='background-color:#BCBD8B;color:white'><td>Number of RuvC1 outliers</td><td>%i</td></tr>" % (len(start_outliers)) +\
    "<tr style='background-color:#717744;color:white'><td>Number of RuvC3 outliers</td><td>%i</td></tr>" % (len(end_outliers)) +\
    "<tr style='background-color:#766153;color:white'><td>Outlier Intersection</td><td>%i</td></tr>" % (len(start_outliers.intersection(end_outliers))) +\
    """
    </table>
"""
dump(hasGoodDomains,"pickles/%s_CorrectDomains.p" % protein)
display(HTML(htmlString)) 

<div class="alert alert-block alert-success">

## Remove chrs that have more than 1 Cas9

</div>

In [ ]:
remove, baseMap, corrected = set(), {}, {}
print(len(hasGoodDomains))
for orfName in hasGoodDomains:
    baseCHR = orfName[:orfName.rfind("_")]
    if baseCHR in corrected:
        try: 
            baseMap[baseCHR].add(orfName)
            baseMap[baseCHR].add(corrected[baseCHR])
        except: baseMap[baseCHR] =set([corrected[baseCHR],orfName])
        remove.add(orfName)
        remove.add(corrected[baseCHR])
    corrected[baseCHR] = orfName
hasGoodDomains = hasGoodDomains.difference(remove)
dump(hasGoodDomains,"pickles/%s_CorrectDomains.p" % protein)
print(len(hasGoodDomains))
casOperons.getRepSeqs(hasGoodDomains,"proteins/All_%s-Like-filtered.faa" %(protein),"proteins/All_%s-Like.faa" % (protein))
dump(casOperons, "pickles/%s_Operons.p" % protein)

<div class="alert alert-block alert-success">

## Cluster the sequences by sequence identity to choose representative sequences

</div>

In [ ]:
%%bash -s $protein
cd-hit -i proteins/All_$1-Like-filtered.faa -M 0 -d 0 -c .90 -sc 1 -o proteins/$1-Like-clustered.faa >logs/$1_ClusterLog.log
tail -n 8 logs/$1_ClusterLog.log > logs/clusterInfo
head -n 1 logs/clusterInfo; rm logs/clusterInfo
mv proteins/$1-Like-clustered.faa.clstr clusters/

<div class="alert alert-block alert-success">

## Align the sequences and create a phylogenetic tree

</div>

In [ ]:
%%bash
sbatch ../scripts/hpc/Alignment.sb

<div class="alert alert-block alert-success">

## Read all the assemblies with a Cas9 and a CRISPR array and create a file with only the pseudochromosome containing both

</div>

In [ ]:
repFile = "assemblies/%s_Representative_Assemblies.fasta" % (protien) 
allRepAssemblies = "assemblies/All_%s_Representative_Assemblies.fasta" % (protien) #The new file we are creating
repSeqs = open(repFile,"w")
casRelatedProteins = fasta_index("proteins/%s-Like-clustered.faa" % protien, "fasta")
for protID,rec in casRelatedProteins.items():
    baseID = protID[:protID.rfind("_")]
    nuceRec = allNukSeqs[baseID]
    nuceRec.id = protID
    if not path.exists("assemblies/pseudoChromos/%s.fasta" % (protID)):
        with open("assemblies/pseudoChromos/%s.fasta" % (protID),'w') as fh: write(nuceRec,fh,"fasta")
    write(nuceRec,repSeqs,"fasta")
    locus = casOperons[protID]
    setattr(locus,'seq',nuceRec) #TODO comment this out on re-run
    locus.seq = nuceRec
repSeqs.close()
dump(casOperons,"pickles/%s_Operons.p" % gene)  